## Executor
It executes a command (`Runnable`) in the future - can be using a new `Thread`, a pooled thread or calling thread. It provides a standard means of decoupling *task submission* from *task execution*. The interface is defined as:

In [ ]:
public interface Executor {
    void execute(Runnable command);
}

A very simplistic implementaion of an executor can be:

In [ ]:
// Executes the runnable in the calling thread
class SimpleExecutor implements Executor {

    @Override
    public void execute(Runnable command) {
        command.run();
    }
}

Another executor which runs tasks in new thread can be:

In [ ]:
class ThreadPerTaskExecutor implements Executor {

    @Override
    public void execute(Runnable command) {
        new Thread(command).run();
    }
}

Decoupling submission from execution lets us easily specify, and subsequently change without great difficulty, the *execution policy* for tasks. It defines things sucha as:
- what thread will be used to execute tasks
- what order should tasks be executed (LIFO, FIFO, priority based)
- how many tasks will be executing concurrently
- how many tasks will wait for execution
- if a task has to be rejected, which one should it be?
- action taken before and after executing a task

## ExecutorService
Builds upon `Executor` and provides many additional functionality to manage termination and methods that can produce a `Future` for tracking progress of one or more asynchronous tasks. For lifecycle management, the following methods are available:

In [ ]:
public interface ExecutorService implements Executor {
    void shutdown();
    List<Runnable> shutdownNow();
    boolean isShutdown();
    boolean isTerminated();
    boolean awaitTermination(long timeout, TimeUnit unit) throws InterruptedException;
    // other methods
}

`shutdown` method is meant for graceful shutdown: no new tasks are accepted but previously submitted tasks are allowed to complete—including those that
have not yet begun execution.  
`shutdownNow` method initiates abrupt shutdown: it *attempts* to cancel running tasks, and does not start queued tasks.  

Tasks submitted to an `ExecutorService` after it has been shutdown are handled by the *rejected execution handler*. `ExecutorService` thereby offers a way to cancel tasks: tasks that have been submitted but not yet started can always be cancelled, and tasks that have started can sometimes be cancelled if they are responsive to interruption.

In order to understand task cancellation better, we need to understand a few topics. Java doesn't provide a mechanism for safely forcing a thread to stop what it is doing. Why? Because abruptly stopping a thread is inherently unsafe. Stopping a thread makes it release all its monitors, so any object protected by this monitor may be in an inconsistent state and other objects waiting for lock would now access this object in an inconsistent state. Consider the following example:

In [ ]:
class Shared {
    private int a, b;
    public Shared(int a, int b){
        this.a = a; this.b = b;
    }
    public synchronized void swap(){
        int temp = a;
        a = b; // What if the Thread A having lock on Shared, 
               // is stopped here? We end up in an inconsistent state
        b = temp;
    }
}

Which is why, we need a cooperative approach - a thread can request another thread to terminate. How (or if) the thread terminates depends upon the thread. So, how do we stop a thread?

We can introduce a cancellation flag:

In [ ]:
class PrimeNumbers implements Runnable {
    private final List<Integer> nums = new ArrayList<>();
    private volatile boolean cancelled; // Must be volatile, so that changes are
                                        // visible to other threads
    public void run(){
        while(!cancelled){
            int p = getNextPrime();
            synchronized(this) { // Need synchronization since ArrayList 
                nums.add(p);     // is not thread safe
            }
        }
    }
    
    public void cancel(){ cancelled = true; }
    
    public synchronized List<Integer> get() {
        return new ArrayList<Integer>(nums); 
    }
}

In other scnerios, Interrupts can be more helpful than cancellation flags. For example, if we use `BlockingQueue` (thread safe, blocks till queue is non-empty while fetching; and for space to be available before storing):

In [ ]:
class PrimeNumbers implements Runnable {
    private final BlockingQueue<Integer> nums; // Initialize in constructor
    private volatile boolean cancelled;
    
    public void run(){
        try {
            while(!cancelled){
                nums.put(getNextPrime());  // -- A
            }
        } catch (InterruptedException e) {}
    }
    
    public void cancel(){ cancelled = true; }
    // ...
}

// A consumer method
void consumePrimes() throws InterruptedException {
    BlockingQueue<Integer> q = new ArrayBlockingQueue<>();
    PrimeNumbers primes = new PrimeNumbers(q);
    Thread pN = new Thread(primes);
    pN.start();
    
    try{
        while(needMorePrimes()) {
            consume(q.take()); // take method blocks
        }
    } finally {
        primes.cancel();
    }
}

There is a problem with the above implementation. If consumer is not able to consume faster than the producer is able to produce, the producer thread will get stuck at A. Even if the cancel method is called, it will not cancel execution.  

In this case, interrupts are the way to go. Many methods in Java respond to interruption, for example:
- `Object.wait`
- `Thread.sleep`, `Thread.join`
- Many methods in `java.util.concurrent`

Interrupting a thread which doesn't respond to interruptions has no visible effect:

In [ ]:
public class InfiniteThread extends Thread {
    public void run() {
        while(true)
            System.out.println("Running...");
    }
}

// calling infiniteThrad.interrupt() will have no
// effect since run block has no interruption points

The `Thread` class has the following interruption related methods:
- `interrupt()`: sets thread's interrupt flag to true
- `interrupted()`: returns value of interrupt flag and then resets it to false
- `isInterrupted()`: returns value of interrupt flag

Now, if we re-write the PrimeNumbers class:

In [ ]:
class PrimeNumbers implements Runnable {
    private final BlockingQueue<Integer> nums; // Initialized in constructor
    
    public void run(){
        try {
            while(!Thread.currentThread().isInterrupted()) { // Checking for interrupt flag here
                nums.put(getNextPrime());                    // makes code a bit more efficient
            }
        } catch (InterruptedException e) { // Ignore so that Thread exits }
    }
    
    public void cancel(){ Thread.currentThread().interrupt(); }
    // ...
}

As we'll see ahead, in many cases we have control over tasks, but not threads (as in `ThreadPoolExecutor`). In such cases, we should try to retain interruption status - one way is to propagate exception up the stack (interrupt again).  

**Non-interruptible Blocking:** not all blocking methods are responsive to interruption. For example:
- Synchronous socket I/O: server apps reading and writing to sockets. The read and write methods are not interruptible, but closing the underlying socket unblocks the blocking read and write, by throwing `SocketException`
- Synchronous I/O in `java.nio`
- Lock acquisition: if a thread is waiting to acquire an intrinsic lock, we can't do anything other than waiting. Use the `Lock` class' `lockInterruptibly` method.

In [ ]:
class SocketReader extends Thread {
    private Socket socket;
    private InputStream input;
    
    public SocketReader (Socket socket) {
       this.socket = socket;
       input = socket.getInputStream();
    }
    
    public void run() {
        try{
            byte[] buffer = new byte[BUF_SIZE];
            while(true){
                int c = input.read(buffer);
                if(c < 0){
                    break;
                } else if (c > 0){
                    processBuffer(buffer, c);
                }
            }
        } catch (IOException e) { 
            //... 
        }
    }
    
    public void interrupt(){
        try {
            socket.close();  // This would bring the thread out from being blocked by read
        } catch (IOException e) { 
            //... 
        } finally {
            super.interrupt();
        }
    }
}

### Callable and Future
`Callable` and `Future` provide better mechanics to control lifecycle of a task. `Callable` is a task that returns a result and may throw an exception. It is defined as:

In [1]:
@FunctionalInterface
public interface Callable<V> {
    
    V call() throws Exception;
}

To create a callable which returns nothing, use `Callable<Void>` variant.  

`Future` on the other hand, represents the result of an asynchronous computation. It provides methods to test whether the task has completed or been cancelled, retrieve its result, and cancel the task. It is defined as:

In [ ]:
public interface Future<V> {
    boolean cancel(boolean mayInterruptIfRunning);
    boolean isCancelled();
    boolean isDone();
    V get() throws InterruptedException, ExecutionException;
    V get(long timeout, TimeUnit unit) throws InterruptedException, ExecutionException, TimeoutException;
}

If an exception occurs while processing the task, it is wrapped in `ExecutionException`, the underlying exception can be retreived using `getCause` method. If we cancel a future, we get `CancellationException`, if we are waiting for it to complete:

In [ ]:
ExecutorService executor = Executors.newFixedThreadPool(5);
Future<String> f = executor.submit(() -> {
   throw new RuntimeException("Hola!");
});

executor.shutdown();
try {
    f.get();
} catch (ExecutionException e) {
    System.out.println(e.getCause()); // Hola!
}

The `cancel` method fails if the task has already completed, has already been cancelled, or could not be cancelled for some other reason. If successful, and this task has not started when cancel is called, this task should never run. If the task has already started, then the `mayInterruptIfRunning` parameter determines whether the thread executing this task should be interrupted in an attempt to stop the task. 

`ExecutorService` provides multiple ways to create a `Future` to describe a task:

In [ ]:
public interface ExecutorService implements Executor {
    <T> Future<T> submit(Callable<T> task);
    <T> Future<T> submit(Runnable task, T result);
    Future<?> submit(Runnable task);
    <T> List<Future<T>> invokeAll(Collection<? extends Callable<T>> tasks) throws InterruptedException;
    <T> List<Future<T>> invokeAll(Collection<? extends Callable<T>> tasks, long timeout, TimeUnit unit) throws InterruptedException;
    <T> T invokeAny(Collection<? extends Callable<T>> tasks) throws InterruptedException, ExecutionException;
    <T> T invokeAny(Collection<? extends Callable<T>> tasks, long timeout, TimeUnit unit) 
        throws InterruptedException, ExecutionException, TimeoutException;
    // other methods
}

The `get` method is a blocking call, so how do we act on a future (from a list of multiple futures) as soon as it is available? A crude way would be to get result with timeout 0 and continuously keep polling. This is a crude implementation:

In [ ]:
ExecutorService executorService = Executors.newFixedThreadPool(5);
List<Future<Long>> futureList = new ArrayList<>(5);
for (int i = 0; i < 5; i++) {
    futureList.add(executorService.submit(getRandomSleepCallable()));
}
executorService.shutdown();

while (true) {
    Iterator<Future<Long>> iterator = futureList.iterator();
    while (iterator.hasNext()) {
        Future<Long> future = iterator.next();
        if (future.isDone()) {
            iterator.remove();
        }

        try {
            System.out.println("Completed thread that slept for: " + future.get(0, TimeUnit.SECONDS));
        } catch (TimeoutException e) {
            // Do nothing
        }

    }

    if (futureList.isEmpty()) {
        break;
    }
}

The better way is to use `CompletionService` defined as:

In [ ]:
public interface CompletionService<V> {
    Future<V> submit(Callable<V> task);
    Future<V> submit(Runnable task, V result); // result is what is returned upon successful completion
    Future<V> take() throws InterruptedException; // Retrieves and removes the Future representing the next
                                                  // completed task, waiting if none are yet present.
    Future<V> poll(); // Retrieves and removes the Future representing the next completed task, or null if none are present.
    Future<V> poll(long timeout, TimeUnit unit) throws InterruptedException; // Waits in this case
}

In [ ]:
ExecutorService executorService = Executors.newFixedThreadPool(5);
CompletionService<Long> completionService = new ExecutorCompletionService<>(executorService);
for (int i = 0; i < 5; i++) {
    completionService.submit(getRandomSleepCallable());
}
for (int i = 0; i < 5; i++) {
    System.out.println("Completed thread that slept for: " + completionService.take().get());
}

## Thread Pools
Independent and homogenous tasks are most suited for being distribted to multiple threads in a thread pool. However, the following tasks need special consideration:
- Dependent tasks: those that depend upon other tasks in the thread pool - we need to implicitly create constraints on the execution policy
- Tasks that use `ThreadLocal`: executors are free to use whichever thread from the pool to execute tasks
- Long running tasks.

We can see below how dependent tasks can lead to deadlock. As an example consider two tasks A and B, where B is submitted by  A and A waits for B to get completed. In a single thread executor this will always deadlock.

In [ ]:
// Important to notice that this is a single thread executor
ExecutorService executor = Executors.newSingleThreadExecutor();

public class RenderPageTask implements Callable<String> {
    public String call() throws Exception {
        Future<String> header, footer;
        
        header = executor.submit(new LoadFileTask("header.html"));
        footer = executor.submit(new LoadFileTask("footer.html"));
        
        String body = renderBody();
        
        return header.get() + body + footer.get();
    }
}

Tasks may also depend upon external resources, for example, JDBC Connections. If for example we have 15 tasks being executed and we have only 10 connections in JDBC connection pool, 5 tasks will just be waiting. In this case, we have unnecessarily large number of threads in the Thread pool.

We can also face the reverse situation if we have long running task. Long running task will eventually saturate the pool preventing shorter task from getting executed. This may be a sign to increase thread pool size.

**Thread Pool Size:** depends upon the type of task that will be submitted. There is no exact science dictating thread pool size, we just need to avoid the extremes. Keep the thread count high and all the threads compete for the limited resource. Keep the thread count low and it would lead to low resource utilization due to cores going unused. Typically:
- for compute intensive tasks, $N_{threads} = N_{cpu} + 1$
- for I/O intensive task, we first need to compute wait time vs compute time for task. In this case we use $N_{threads} = N_{cpu}\times U(1 + \frac{W}{C})$ where $U$ is target CPU utilization and $\frac{W}{C}$ is ratio of wait time to compute time

**ThreadPoolExecutor:** is an `ExecutorService` that executes each submitted task using one of possibly several pooled threads. It provides multiple constructors the most important (illustrating its configrations) is:

In [ ]:
public ThreadPoolExecutor(int corePoolSize,    // maintain these many threads (even if there are no tasks)
                          int maximumPoolSize, // maximum number of threads can be active as once
                          long keepAliveTime,  // thread alive for more than this time become eligible to be terminated
                          TimeUnit unit,     
                          BlockingQueue<Runnable> workQueue, // Holds tasks awaiting execution
                          ThreadFactory threadFactory,
                          RejectedExecutionHandler handler) { /* ... */}

The `BlockingQueue` argument holds tasks that are waiting to be executed. There are three approaches:
- unbounded queue: keep adding newly arriving tasks to the queue. Queue will expand as the tasks increase.
- bounded queue: fixed room. What happens when the queue is full and new tasks arrive? Depends upon the saturation policy. In this case the queue  size and threadpool size must be in sync. 
- synchronous handoff: bypass queueing entirely and instead handoff that task to the worker thread. How does this work if all the worker threads are busy? If core pool size is less than max pool size, new thread will be created, else it depends upon the saturation policy.

Java offers multiple implementations:
- `LinkedBlockingQueue` : unbounded queue
- `ArrayBlockingQueue` : bounded queue, need to specify its size
- `PriorityBlockingQueue` : unbounded, same priority as `PriorityQueue`
- `SynchronousQueue` : not a pure queue, contains a single item

The below example illustrates the working of blocking queues:

In [ ]:
public class BlockingQueueDemo {
    public static void main(String[] args) {
        BlockingQueue<Integer> queue = new LinkedBlockingQueue<>();

        Producer p = new Producer(queue);
        Consumer c = new Consumer(queue);

        new Thread(p).start();
        new Thread(c).start();
    }
}

class Producer implements Runnable {
    private BlockingQueue<Integer> queue;

    public Producer(BlockingQueue<Integer> queue) {
        this.queue = queue;
    }

    @Override
    public void run() {
        // Put 100 even numbers in queue
        for (int i = 0; i < 100; i++) {
            try {
                queue.put(i * 2);
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }
    }

}

class Consumer implements Runnable {
    private BlockingQueue<Integer> queue;

    public Consumer(BlockingQueue<Integer> queue) {
        this.queue = queue;
    }

    @Override
    public void run() {
        // Consume 100 integers
        for (int i = 0; i < 100; i++) {
            try {
                System.out.println(queue.take());
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }
    }

}

**Saturation Policy:** when the work queue fills up, saturation polcy kicks in (also kicks in when task is submitted to an `Executor` that has been shut down). The saturation policy of a `ThreadPoolExecutor` is set by calling `setRejectedExecutionHandler`. Several implementations of `RejectedExecutionHandler` are available:
- `AbortPolicy`: causes execute to throw unchecked `RejectedExecutionException`
- `DiscardPolicy`: silently discards the submitted task
- `DiscardOldestPolicy`: discards the task that was going to be executed next and queue the newly added task
- `CallerRunsPolicy`: neither discard, nor throw the exception. Execute the submitted task not in the threadpool, but in the thread that submitted the task

**Thread Factories:** is defined as:

In [ ]:
public interface ThreadFactory {
    Thread newThread(Runnable r);
}

the default thread factory used by `ThreadPoolExecutor` creates a new, nondaemon thread with no special configuration. Specifying a `ThreadFactory` helps us specify how exactly a thread (to be added in threadpool) will be created. Why do want to do that?
- increase thread priority
- add custom logging
- give threads custom name, etc

In [ ]:
public class CustomThreadFactory implements ThreadFactory {
    private final String poolName;
    
    public CustomThreadFactory(String poolName) {
        this.poolName = poolName;
    }
    
    public Thread newThread(Runnable r) {
        return new CustomThread(r, poolName);
    }
}

class CustomThread extends Thread {
    public static final String DEFAULT_NAME = "CustomThread";
    private static volatile boolean debugLifecycle = false;
    private static final AtomicInteger created = new AtomicInteger();
    private static final AtomicInteger alive = new AtomicInteger();
    private static final Logger logger = Logger.getAnonymousLogger();
    
    public CustomThread(Runnable r){
        this(r, DEFAULT_NAME);
    }
    
    public CustomThread(Runnable r, String poolName){
        super(r, name + "_" + created.incrementAndGet());
        setUncaughtExceptionHandler(
            new Thread.UncaughtExceptionHandler() {
                public void uncaughtException(Thread t, Throwable e) {
                    loggger.log(Level.SEVERE, "Uncaught in thread " + t.getNme(), e);
                }
            }
        );
    }
        
    public void run() {
        boolean debug = debugLifecycle;
        if(debug) logger.log(Level.FINE, "Created " + getName());
        try {
            alive.incrementAndGet();
            super.run();
        } finally {
            alive.decrementAndGet();
            if(debug) logger.log(Level.FINE, "Exiting " + getName());
        }
    }
    
    public static int getThreadsCreated() { return created.get(); }
    public static int getThreadsAlive() { return alive.get(); }
    public static int getDebug() { return debugLifecycle; }
    public static int setDebug(boolean b) { debugLifecycle = b }
}

The `Executors` class provides multiple different variants of `ExecutorService` utilizing threadpools:

In [ ]:
// --- Fixed Thread Pool ---
ThreadPoolExecutor fTP = (ThreadPoolExecutor) Executors.newFixedThreadPool(10);
/* Equivalent to:
new ThreadPoolExecutor(nThreads, nThreads,
                       0L, TimeUnit.MILLISECONDS,
                       new LinkedBlockingQueue<Runnable>()); */

// --- Cached Thread Pool ---
ThreadPoolExecutor cTP = (ThreadPoolExecutor) Executors.newCachedThreadPool();
/* Equivalent to:
new ThreadPoolExecutor(0, Integer.MAX_VALUE,
                       60L, TimeUnit.SECONDS,
                       new SynchronousQueue<Runnable>()); */

// --- Single Thread Executor ---
ThreadPoolExecutor sTP = (ThreadPoolExecutor) Executors.newSingleThreadExecutor();
/* Equivalent to:
new ThreadPoolExecutor(1, 1,
                       0L, TimeUnit.MILLISECONDS,
                       new LinkedBlockingQueue<Runnable>())) */

## CountdownLatch
Provides one way to wait for all the tasks to get completed before moving forward. `CountDownLatch` has a counter field, which we can decrement as we require. We can then use it to block a calling thread until it’s been counted down to zero.

In [ ]:
public class CountdownLatchDemo {
    public static void main(String[] args) throws InterruptedException {
        ExecutorService service = Executors.newFixedThreadPool(5);
        CountDownLatch latch = new CountDownLatch(10);
        Stream.generate(() -> new Worker(latch))
                .limit(10)
                .forEach(worker -> service.execute(worker));
        
        // Oracle's suggested way of terminating an executor service
        service.shutdown();
        try {
            if (!service.awaitTermination(10000, TimeUnit.MILLISECONDS)) {
                service.shutdownNow();
            }
        } catch (InterruptedException e) {
            service.shutdownNow();
        }

        latch.await();
        System.out.println("All tasks have completed execution");
    }

    static class Worker implements Runnable {
        private CountDownLatch latch;

        public Worker(CountDownLatch latch) {
            this.latch = latch;
        }

        @Override
        public void run() {
            try {
                Thread.sleep(ThreadLocalRandom.current().nextInt(500, 5000));
                System.out.println("Completed execution");
                latch.countDown();
            } catch (InterruptedException e) {
                throw new RuntimeException(e);
            }
        }
    }
}

There is one problem with the above approach. What if `run` throws exception before `countdown()` method is called? Our program will wait indefinitely. So either put `countDown` in `finally` or use the other version of `await` which accepts a timeout argument.